In [1]:
import os
import numpy as np
# from time import time, strptime
import cv2
from tqdm import tqdm
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
import sys

In [2]:
def mkdir(path):
    if os.path.isdir(path):
        print("Directory '"+path+"' already exists")
    else:
        try:
            os.makedirs(path)
        except OSError:
            print("'"+path+"' is an invalid path")

In [3]:
def mazza_dataset(video_file, frame_rate=1, square=True, new_size=None):
    video = cv2.VideoCapture(video_file)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count//fps
    instants = np.linspace(0,duration,num=duration*frame_rate,endpoint=False)
    frame_idxs = round(fps)*instants.reshape(-1,1)
    dataset = []
    for instant in instants:
        video.set(cv2.CAP_PROP_POS_MSEC,instant*1000)
        errRead, frame = video.read()
        if frame is None:
            print('Instant {} returns a null frame',instant)
        else:
            if square is True:
                border = int((frame.shape[1] - frame.shape[0])/2)
                frame = frame[:, border : (frame.shape[1]-border), :]
            if new_size is not None:
                frame = cv2.resize(frame, new_size) 
            dataset.append(frame)
    dataset = np.stack(dataset)
    assert(dataset.shape[0] == len(frame_idxs))
    return dataset, frame_idxs
# video.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
# int(video.get(cv2.CAP_PROP_FRAME_COUNT)) #CAP_PROP_FRAME_COUNT CAP_PROP_POS_MSEC CAP_PROP_POS_FRAMES

In [4]:
%%time
video_file = '../../Videos/PL0138/HD_C_pl0138_VOL006.mp4'
frame_rate = 2
square = True
new_size = (224,224)

dataset, frame_idxs = mazza_dataset(video_file, frame_rate, new_size=new_size)

/anaconda3/envs/opencvenv/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


KeyboardInterrupt: 

In [5]:
interact(interactive_print, index = (0,len(dataset)))

NameError: name 'interactive_print' is not defined

In [ ]:
print('dataset size: {}MB'.format(round(sys.getsizeof(dataset)/1e6)))

In [ ]:
target_path = '../../visualizacao_petrobras-master/dataset/video06'
dataset_filename = 'dataset06_framerate2.h5'
frames_filename = 'frameidxs06.npy'

mkdir(target_path)
# np.save(os.path.join(target_path,dataset_filename),dataset)
h5f = h5py.File(os.path.join(target_path,dataset_filename), 'w')
h5f.create_dataset('X_te', data=dataset)
h5f.close()
# np.save(os.path.join(target_path,frames_filename),frame_nums)

In [7]:
predfile = '../../mazza/dataset/video06/predictions06_weighted.h5'
idxfile = '../../mazza/dataset/video06/frameidxs06.npy'
targetfile = '../../mazza/dataset/video06/predictions.npy'

with h5py.File(predfile, 'r') as hf:
    pred = hf['pred'][:]
idx = np.rint(np.load(idxfile).reshape(-1,1))

# print(idx.shape,pred.shape)
np.save(targetfile,np.concatenate((idx,pred),axis=1))

In [8]:
x = np.load('../../mazza/dataset/video06/predictions06.npy')
x

array([[0.00000000e+00, 1.67865437e-02, 1.44045437e-02],
       [3.00000000e+01, 8.64090696e-02, 2.52069049e-02],
       [6.00000000e+01, 1.33047208e-01, 6.13176003e-02],
       ...,
       [5.38560000e+04, 4.09950167e-02, 5.25340959e-02],
       [5.38860000e+04, 2.96453219e-02, 1.65397078e-02],
       [5.39160000e+04, 1.77728217e-02, 2.44723279e-02]])

In [ ]:
import glob

glob.glob('../LR/*')